In [1]:
import os

import numpy as np
import torch

import datasets
from preprocessed_dataset import DecisionTransformerPreprocessedDataset, UnwrapCollator
import torch.utils.data
from transformers import TrainingArguments, Trainer
from decision_transformer import DecisionTransformerConfig, DecisionTransformerModel

import snowietxt_processor


In [2]:
os.environ["WANDB_DISABLED"] = "true"
torch.backends.cuda.matmul.allow_tf32 = True

### Step 4: Defining a custom DataCollator for the transformers Trainer class

In [3]:
dataset = snowietxt_processor.create_dataset()
dataset = datasets.Dataset.from_dict(dataset)

Opening games\game0
Opening games\game1
Opening games\game2
Opening games\game3
Opening games\game4
Opening games\game5
Opening games\game6
Opening games\game7
Opening games\game8
Opening games\game9
Opening games\game10
Opening games\game11
Opening games\game12
Opening games\game13
Opening games\game14
Opening games\game15
Opening games\game16
Opening games\game17
Opening games\game18
Opening games\game19
Opening games\game20
Opening games\game21
Opening games\game22
Opening games\game23
Opening games\game24
Opening games\game25
Opening games\game26
Opening games\game27
Opening games\game28
Opening games\game29
Opening games\game30
Opening games\game31
Opening games\game32
Opening games\game33
Opening games\game34
Opening games\game35
Opening games\game36
Opening games\game37
Opening games\game38
Opening games\game39
Opening games\game40
Opening games\game41
Opening games\game42
Opening games\game43
Opening games\game44
Opening games\game45
Opening games\game46
Opening games\game47
Op

100%|██████████| 5105/5105 [00:02<00:00, 1981.27it/s]


In [ ]:
dataset = DecisionTransformerPreprocessedDataset(dataset, max_len=10, batch_size=64)

Preprocessing dataset:  73%|███████▎  | 350/480 [06:36<00:54,  2.38batch/s]

In [ ]:
# we want to preprocess the dataset using the data collator we defined above (it also handles the batches
# right now, just test to see how many batches we get using the collator and batch size of 64
# from torch.utils.data import DataLoader
#
# collator = DecisionTransformerGymDataCollator(dataset)
# dataloader = DataLoader(dataset, batch_size=64, collate_fn=collator)


In [ ]:
# states, actions, rewards, returns_to_go, timesteps, attention_mask = [], [], [], [], [], []
#
# num_batches = 0
#
# for i in range(10 // DecisionTransformerGymDataCollator.max_len): # TODO: figure out the average length of an episode
#     for batch in tqdm(dataloader, total=len(dataloader)):
#         states.append(batch["states"])
#         actions.append(batch["actions"])
#         rewards.append(batch["rewards"])
#         returns_to_go.append(batch["returns_to_go"])
#         timesteps.append(batch["timesteps"])
#         attention_mask.append(batch["attention_mask"])
#
#         num_batches += 1


In [ ]:
# calculate how much memory the states take up
# states_view = torch.cat(states, dim=0)
# actions_view = torch.cat(actions, dim=0)
#
# print(f"States shape: {states_view.shape}")
# print(f"Actions shape: {actions_view.shape}")
#
# # calculate how much memory the states take up
# print(f"States memory: {states_view.element_size() * states_view.nelement() / 1e6} MB")
# print(f"Actions memory: {actions_view.element_size() * actions_view.nelement() / 1e6} MB")

### Step 5: Extending the Decision Transformer Model to include a loss function

In order to train the model with the 🤗 trainer class, we first need to ensure the dictionary it returns contains a loss, in this case L-2 norm of the models action predictions and the targets.

In [ ]:
config = DecisionTransformerConfig(state_dim=dataset.state_dim, act_dim=dataset.act_dim)
model = DecisionTransformerModel(config)

### Step 6: Defining the training hyperparameters and training the model
Here, we define the training hyperparameters and our Trainer class that we'll use to train our Decision Transformer model.

This step takes about an hour, so you may leave it running. Note the authors train for at least 3 hours, so the results presented here are not as performant as the models hosted on the 🤗 hub.

In [ ]:
training_args = TrainingArguments(
    output_dir="output/",
    remove_unused_columns=False,
    num_train_epochs=100,
    per_device_train_batch_size=1,
    learning_rate=1e-4,
    weight_decay=1e-4,
    warmup_ratio=0.1,
    optim="adamw_torch",
    max_grad_norm=0.25,
    tf32=True,
    fp16=True,
    dataloader_pin_memory=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=UnwrapCollator(),
)

trainer.train()

In [ ]:
from torch.utils.data import DataLoader

# create a dataloader for evaluation
eval_dataloader = DataLoader(dataset, batch_size=1, collate_fn=dummy_collator)

# get one batch from the dataloader and run it through the model
batch = next(iter(eval_dataloader))
model.cuda()
model.eval()

with torch.no_grad():
    output = model.original_forward(**batch)

print(output['action_preds'][0].argmax(dim=-1))
print(batch['actions'][0])
